In [43]:
import praw
import pandas as pd
from tqdm import tqdm
import re
from bs4 import BeautifulSoup

Create a reddit instance to scrap data

In [2]:
reddit = praw.Reddit(
    client_id="lvMebPoBq4Am77B71uC3Pw",
    client_secret="_vZg-umYkXpIRZoI_klQP3eYHDqOVA",
    password="argumentgate",
    user_agent="testscript by u/fakebot3",
    username="SaanJatt1",
)

Version 7.5.0 of praw is outdated. Version 7.6.0 was released Tuesday May 10, 2022.


In [3]:
print(reddit.user.me())

SaanJatt1


Get all the submissions since the 2005 from DeltaLog reddit 

In [4]:
import datetime as dt
import praw
from psaw import PushshiftAPI

api = PushshiftAPI(reddit)

start_epoch=int(dt.datetime(2005, 1, 1).timestamp()) # Could be any date

submissions_generator = api.search_submissions(after=start_epoch, subreddit='DeltaLog', limit=100000) # Returns a generator object
submissions = list(submissions_generator) # You can then use this, store it in mongoDB, etc.

/gpfs/fs1/home/j/jcaunedo/umar1/nlp/lib64/python3.6/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


Get all the submission and delta data

In [69]:
delta_data = []
for sub in tqdm(submissions):
    
    try:
        # Raw html text of submission
        text = sub.selftext_html

        soup = BeautifulSoup(text,'html.parser')
        
        # Find statement id
        state_id = soup.find('a')['href'].replace("/r/changemyview/comments/","").split('/')[0]
        
        for each in soup.find_all('li'):
            
            # get delta description and add it to data
            
            desc = each.find_all('a')[1].text
            delta_data.append([state_id,desc])
    except:
        continue

100%|██████████| 33657/33657 [00:40<00:00, 832.90it/s]


In [72]:
# Save the data as a csv file

delta_data = pd.DataFrame(delta_data,columns=["statement_id","arguement"])

delta_data["delta"] = 1

delta_data.to_csv('delta.csv')